In [1]:
%run ~/common_init.py
import flamedisx as fd
import tensorflow as tf
#fd.utils.FLOAT_TYPE = tf.float64
fd.float_type()

Active python: /home/jelle/miniconda3/envs/py37/bin/python


tf.float32

In [2]:
rn_data = pd.read_csv('data_Rn220_lowenergy.csv')
fd.SR0ERSource.annotate_data(rn_data)
rn_data = rn_data[rn_data['e_vis'] < 10].copy()
rn_data = rn_data[['s1', 's2'] + ['x_observed', 'y_observed', 'x', 'y', 'z', 'drift_time']]
rn_data = rn_data[::32]

In [8]:
len(rn_data)

21

In [3]:
ll = fd.LogLikelihood(
    sources=dict(er=fd.SR0ERSource),
    batch_size=5,
    free_rates='er',
    data=rn_data,
    elife=(300e3, 500e3, 3),
    erqy_a=(5., 25., 2),
    erqy_b=(-29., -26., 2),
    erqy_c=(31.9, 32.8, 2),
    g2=(25., 35., 2),
    n_trials=int(2e5))
len(rn_data), len(rn_data) / ll.mu_itps['er']().numpy()

Estimating mus: 100%|██████████| 5/5 [00:28<00:00,  6.05s/it]


(21, 0.02419828720654397)

In [ ]:
for pname in ll.param_names:
    if pname.endswith('_rate_multiplier'):
        continue
    start, stop, n = ll.param_specs[pname]
    x = np.linspace(start, stop, 100, dtype=np.float32)
    y = np.array([ll.mu_itps['er'](**{pname: tf.constant(q)}) for q in x])
    plt.plot(x, y)
    plt.title(pname)
    plt.show()

In [5]:
guess = ll.guess().numpy()
guess[0] = 0.02 #* 32

In [6]:
%%time
bestfit = ll.bestfit(guess)

CPU times: user 6min 38s, sys: 9.04 s, total: 6min 47s
Wall time: 2min 15s


In [7]:
%%prun -D bla.prof
bestfit = ll.bestfit(guess)

 
*** Profile stats marshalled to file 'bla.prof'. 


In [8]:
%%time
bestfit = ll.bestfit(guess)

CPU times: user 6min 41s, sys: 21.6 s, total: 7min 3s
Wall time: 2min 20s


In [9]:
%%time
bestfit = ll.bestfit(guess)

E0712 15:44:04.399806 140412421769024 ultratb.py:149] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jelle/miniconda3/envs/py37/lib/python3.7/site-packages/IPython/core/magics/execution.py", line 1291, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 1, in <module>
  File "/home/jelle/software/flamedisx/flamedisx/inference.py", line 185, in bestfit
    res = optimizer(objective, x_norm, **kwargs)
  File "/home/jelle/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow_probability-0.7.0rc0-py3.7.egg/tensorflow_probability/python/optimizer/lbfgs.py", line 260, in minimize
    parallel_iterations=parallel_iterations)[0]
  File "/home/jelle/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py", line 2457, in while_loop_v2
    return_same_structure=True)
  File "/home/jelle/miniconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py", line 2689, in while_loop
    loop_vars = body(*loop_vars)
  File "/home/jelle/miniconda3/envs/py37/lib/python3.7/sit

KeyboardInterrupt: 

In [12]:
%%time
inv_hess = ll.inverse_hessian(bestfit)

Computing hessian: 100%|██████████| 9/9 [02:36<00:00, 18.62s/it]

CPU times: user 6min 4s, sys: 3.07 s, total: 6min 7s
Wall time: 2min 36s


In [15]:
# %%time
# inv_hess = ll.inverse_hessian(bestfit, save_ram=False)

In [17]:
ll.summary(bestfit, inverse_hessian=inv_hess)

er_rate_multiplier: 0.0207 +- 0.00347
elife: 4.37e+05 +- 3.28e+04
erqy_a: 14.3 +- 17.9
erqy_b: -31.9 +- 4.62
erqy_c: 32.7 +- 2.21
g2: 32.4 +- 1.15
Correlation matrix:
                    er_rate_multiplier  elife     erqy_a  erqy_b  erqy_c  \
er_rate_multiplier               1.000 -0.046  9.547e-02   0.013  -0.002   
elife                           -0.046  1.000  7.968e-03   0.370  -0.630   
erqy_a                           0.095  0.008  1.000e+00   0.450  -0.559   
erqy_b                           0.013  0.370  4.497e-01   1.000  -0.652   
erqy_c                          -0.002 -0.630 -5.593e-01  -0.652   1.000   
g2                              -0.021 -0.137  9.493e-04   0.010  -0.283   

                           g2  
er_rate_multiplier -2.117e-02  
elife              -1.365e-01  
erqy_a              9.493e-04  
erqy_b              9.887e-03  
erqy_c             -2.827e-01  
g2                  1.000e+00  


er_rate_multiplier: 0.021 +- 0.00353
elife: 4.37e+05 +- 3.27e+04
erqy_a: 12.7 +- 17.5
erqy_b: -31.6 +- 4.61
erqy_c: 32.8 +- 2.23
g2: 32.3 +- 1.31
Correlation matrix:
                    er_rate_multiplier  elife  erqy_a  erqy_b  erqy_c     g2
er_rate_multiplier               1.000 -0.035   0.116   0.028  -0.022 -0.017
elife                           -0.035  1.000   0.022   0.364  -0.613 -0.156
erqy_a                           0.116  0.022   1.000   0.447  -0.554  0.010
erqy_b                           0.028  0.364   0.447   1.000  -0.638  0.010
erqy_c                          -0.022 -0.613  -0.554  -0.638   1.000 -0.325
g2                              -0.017 -0.156   0.010   0.010  -0.325  1.000


In [ ]:
ll.summary(bestfit, inverse_hessian=inv_hess, save_ram=False)

In [24]:
guess = ll.guess().numpy()
guess[0] = 0.01
bestfit = ll.bestfit(guess)
inv_hess = ll.inverse_hessian(bestfit)
ll.summary(bestfit, inverse_hessian=inv_hess)

Computing hessian: 100%|██████████| 6/6 [01:23<00:00, 15.83s/it]

er_rate_multiplier: 0.00921 +- 0.00236
elife: 4.34e+05 +- 4.84e+04
erqy_a: 14.5 +- 27.4
erqy_b: -32.1 +- 6.98
erqy_c: 32.8 +- 3.33
g2: 32.4 +- 1.76
Correlation matrix:
                    er_rate_multiplier  elife  erqy_a  erqy_b  erqy_c  \
er_rate_multiplier           1.000e+00 -0.003   0.146   0.064  -0.080   
elife                       -2.567e-03  1.000   0.006   0.362  -0.621   
erqy_a                       1.459e-01  0.006   1.000   0.449  -0.564   
erqy_b                       6.420e-02  0.362   0.449   1.000  -0.648   
erqy_c                      -8.024e-02 -0.621  -0.564  -0.648   1.000   
g2                           6.193e-04 -0.142  -0.001   0.008  -0.283   

                           g2  
er_rate_multiplier  6.193e-04  
elife              -1.424e-01  
erqy_a             -1.292e-03  
erqy_b              8.372e-03  
erqy_c             -2.832e-01  
g2                  1.000e+00  


In [ ]:
er_source_s = fd.SR0ERSource()
er_source_s.set_data(rn_data)
n_trials = int(1e5)

sim_guess = er_source_s.simulate(n_trials,
                                **ll.params_to_dict(guess))
sim_bf = er_source_s.simulate(n_trials, 
                              **ll.params_to_dict(bestfit.numpy()))

bins = np.linspace(0, 11, 10)
for d_sim in sim_guess, sim_bf:
    Histdd(d_sim, dimensions=(
        ('e_vis', np.linspace(0, 11, 50)),
        ('electron_produced_mle', np.linspace(0, 350, 50)),
    )).plot()
    d = rn_data
    plt.scatter(d['e_vis'], d['electron_produced_mle'], c='white', s=5)
    plt.show()


# OLD

In [9]:
# @tf.function
# def bla(x):
#     y = ll.minus_ll(x)
#     #grads = tf.gradients(y, x)
#     return tf.hessians(y, x)

# logdir = '/tmp/summaries/bla'
# writer = tf.summary.create_file_writer(logdir)
# tf.summary.trace_on(graph=True, profiler=True)
# # Forward pass
# bla(bestfit)
# with writer.as_default():
#     tf.summary.trace_export(name="bla", step=0, profiler_outdir=logdir)